## install mecab on mac
```
brew install mecab mecab-ipadic  
pip install mecab-python3
```

## install neologd
```
git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
cd mecab-ipadic-neologd
./bin/install-mecab-ipadic-neologd -n
```

## get Japanese articles
livedoor ニュースコーパス

```
wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
tar xvzf ldcc-20140209.tar.gz
```

In [1]:
from urllib import request 
import logging
from pathlib import Path
import numpy as np
import re
import MeCab
from gensim import corpora, models
import random

try:
    if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
        print("tqdm is notebook version")
        from tqdm import tqdm_notebook as tqdm
    else:
        raise RuntimeError
except (NameError, RuntimeError):
    from tqdm import tqdm

tqdm is notebook version


In [2]:
mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")

In [3]:
[line.split("\t") for line in mecab.parse("認めたくないものだな。自分自身の若さ故の過ちというものを。").split("\n")]

[['認め', 'ミトメ', '認める', '動詞-自立', '一段', '連用形'],
 ['たく', 'タク', 'たい', '助動詞', '特殊・タイ', '連用テ接続'],
 ['ない', 'ナイ', 'ない', '助動詞', '特殊・ナイ', '基本形'],
 ['もの', 'モノ', 'もの', '名詞-非自立-一般', '', ''],
 ['だ', 'ダ', 'だ', '助動詞', '特殊・ダ', '基本形'],
 ['な', 'ナ', 'な', '助詞-終助詞', '', ''],
 ['。', '。', '。', '記号-句点', '', ''],
 ['自分自身', 'ジブンジシン', '自分自身', '名詞-固有名詞-一般', '', ''],
 ['の', 'ノ', 'の', '助詞-連体化', '', ''],
 ['若さ故の過ち', 'ワカサユエノアヤマチ', '若さ故の過ち', '名詞-固有名詞-一般', '', ''],
 ['という', 'トイウ', 'という', '助詞-格助詞-連語', '', ''],
 ['もの', 'モノ', 'もの', '名詞-非自立-一般', '', ''],
 ['を', 'ヲ', 'を', '助詞-格助詞-一般', '', ''],
 ['。', '。', '。', '記号-句点', '', ''],
 ['EOS'],
 ['']]

In [4]:
[line.split("\t") for line in mecab.parse("今日もがんばるぞい！").split("\n")]

[['今日', 'キョウ', '今日', '名詞-副詞可能', '', ''],
 ['も', 'モ', 'も', '助詞-係助詞', '', ''],
 ['がん', 'ガン', 'がん', '名詞-一般', '', ''],
 ['ばる', 'バル', 'バル', '名詞-一般', '', ''],
 ['ぞい', 'ゾイ', 'ぞい', '名詞-接尾-一般', '', ''],
 ['！', '！', '！', '記号-一般', '', ''],
 ['EOS'],
 ['']]

In [5]:
[line.split("\t") for line in mecab.parse("今日も一日がんばるぞい").split("\n")]

[['今日も一日がんばるぞい', 'キョウモイチニチガンバルゾイ', '今日も一日がんばるぞい', '名詞-固有名詞-一般', '', ''],
 ['EOS'],
 ['']]

In [6]:
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt")
stopwords = [line.decode("utf-8").strip() for line in res]
print(stopwords[:3])

['あそこ', 'あたり', 'あちら']


In [7]:
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/English.txt")
stopwords += [line.decode("utf-8").strip() for line in res]
print(stopwords[-3:])

["you've", 'z', 'zero']


In [8]:
class Tokenizer:
    def __init__(self, stopwords, parser=None, include_pos=None, exclude_posdetail=None, exclude_reg=None):
    
        self.stopwords = stopwords
        self.include_pos = include_pos if include_pos else  ["名詞", "動詞", "形容詞"]
        self.exclude_posdetail = exclude_posdetail if exclude_posdetail else ["接尾", "数"]
        self.exclude_reg = exclude_reg if exclude_reg else r"$^"  # no matching reg
        if parser:
            self.parser = parser
        else:
            mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")
            self.parser = mecab.parse
            

    def tokenize(self, text, show_pos=False):
        text = re.sub(r"https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+", "", text)    #URL
        text = re.sub(r"\"?([-a-zA-Z0-9.`?{}]+\.jp)\"?" ,"", text)  # xxx.jp 
        text = text.lower()
        l = [line.split("\t") for line in self.parser(text).split("\n")]
        res = [
            i[2] if not show_pos else (i[2],i[3]) for i in l 
                if len(i) >=4 # has POS.
                    and i[3].split("-")[0] in self.include_pos
                    and i[3].split("-")[1] not in self.exclude_posdetail
                    and not re.search(r"(-|−)\d", i[2])
                    and not re.search(self.exclude_reg, i[2])
                    and i[2] not in self.stopwords          
            ]
        return res

In [9]:
t = Tokenizer(stopwords + ["…。"] , mecab.parse, exclude_reg=r"\d(年|月|日)")

In [10]:
t.tokenize("認めたくないものだな。自分自身の若さ故の過ちというものを。")

['認める', '自分自身', '若さ故の過ち']

In [11]:
t.tokenize("認めたくないものだな。自分自身の若さ故の過ちというものを。", show_pos=True)

[('認める', '動詞-自立'), ('自分自身', '名詞-固有名詞-一般'), ('若さ故の過ち', '名詞-固有名詞-一般')]

In [12]:
t.tokenize("-100", show_pos=True)

[]

In [13]:
t.tokenize("食べられる。", show_pos=True)

[('食べる', '動詞-自立')]

In [14]:
doc_path = "./text/"
doc_dir = Path(doc_path)
dirs = [i for i in doc_dir.iterdir() if i.is_dir()]
dirs

[PosixPath('text/movie-enter'),
 PosixPath('text/it-life-hack'),
 PosixPath('text/kaden-channel'),
 PosixPath('text/topic-news'),
 PosixPath('text/livedoor-homme'),
 PosixPath('text/peachy'),
 PosixPath('text/sports-watch'),
 PosixPath('text/dokujo-tsushin'),
 PosixPath('text/smax')]

In [15]:
articles = [a for categ in dirs for a in categ.iterdir()]
random.shuffle(articles)

print(articles[:3])
num_docs = len(articles)
print("# of docs:", num_docs)

[PosixPath('text/peachy/peachy-6755396.txt'), PosixPath('text/livedoor-homme/livedoor-homme-5927832.txt'), PosixPath('text/dokujo-tsushin/dokujo-tsushin-6313075.txt')]
# of docs: 7376


In [16]:
num_docs = len(articles)
num_docs

7376

In [17]:
class Doc_manager():
    def __init__(self, docs):
        self.docs = docs
        
    def read_doc(self, doc_id):
        with self.docs[doc_id].open() as f:
            print(f.read())

In [18]:
dm = Doc_manager(articles)

In [19]:
dm.read_doc(0)

http://news.livedoor.com/article/detail/6755396/
2012-07-15T18:00:00+0900
男が好む清純派の演じ方／「結婚はムリ」と男に見切りをつけた瞬間など−【恋愛】週間ランキング 
Peachyでも大人気の「恋愛」をテーマにした記事の週間ランキングです！2012年7月5日〜7月11日の間にPeachyのアプリでみんなが読んだ恋愛記事TOP5をお届けします。

第1位：男はビッチよりもピュアが好き！清純派っぽく演じる方法8選

男性は、基本的に自分より恋愛経験値の低い女性を好む。著者も最近出会った男性から、これまで付き合った人数を訊かれ「うーん、2人かな」と答えてみると「26歳で2人！　意外と少ないんだね」とニンマリされ、かなり喜ばれてしまった。やはりピュアなほうがモテる。
そこで、男性への会話のなかで“ピュアっぽく見せる”テクニックをご紹介しよう。

第2位：「この男とは絶対結婚できない！」彼氏と別れを決意した瞬間5選

女性が彼氏に見切りをつけるときとは、どんなときなのでしょうか？　筆者の周りにいる多くの女性は、結婚を考え出したときという意見でした。
そこで、「この男とは、絶対に結婚できない！」と思う条件を聞いてみました。

第3位：彼氏とのマンネリを引き起こす女性7の行動

最近彼からの愛情を昔より感じなくなった…と嘆き節な女性、その原因考えたことありますか？
今回は、そんなマンネリ状態を引き起こす女性の行動をまとめてみました。

第4位：許されるのは綾瀬はるかだけ！男が幻滅する「干物女」の特徴6つ

ドラマや映画がヒットした『ホタルノヒカリ』。おかげで干物女という言葉が一般化しましたが、そんな今でも、男性は女性に対してきちんとしていて欲しいと思っているようです。
オーディション等で女性を見ぬかなければいけない機会の多い、放送作家の白武ときおさんは以下のように語ります。「男性は意外と細かいところも見ています。だらしない女の人が好きな男性はまずいないので気を抜くと大変なことになりますよ」

第5位：男性に聞く！　「絶対裏切れない」と感じた彼女の行動

付き合っている人に、浮気をされたら……。余計な心配とはわかっていても、つい疑心暗鬼になって「浮気しないでね」なんて言っては彼を困らせていませんか？　彼らが「

In [20]:
def get_bow(doc_id):
    with articles[doc_id].open() as f:
        f.readline()
        f.readline()
        return t.tokenize(f.read())

In [21]:
get_bow(0)[:10]

['好む', '清純派', '演じる', '結婚', 'ムリ', '見切り', 'つける', '瞬間', '恋愛', '週間ランキング']

In [22]:
docs = []
for i, a in enumerate(tqdm(articles)):
    with a.open() as f:
        f.readline()
        f.readline()
        w = t.tokenize(f.read())
        docs.append(w)

HBox(children=(IntProgress(value=0, max=7376), HTML(value='')))

In [23]:
docs[0][:10]

['好む', '清純派', '演じる', '結婚', 'ムリ', '見切り', 'つける', '瞬間', '恋愛', '週間ランキング']

In [24]:
d = corpora.Dictionary(docs)

In [25]:
len(d)

84628

In [33]:
d[10000]

'探る'

In [35]:
d.token2id["探る"]

10000

In [36]:
d.doc2bow(["終","終","短い"])

[(2980, 1), (22023, 2)]

stopwordsで排除しきれなかった邪魔な単語を処理

In [37]:
d.filter_extremes(no_below=5, no_above=0.2)
# nob_below以下の個数の単語を無視
# no_aboveの割合以上に出てくる単語を無視

d.compactify()
# idを振り直してコンパクトにする。

In [38]:
dic_num = len(d)
dic_num

23513

In [39]:
d[0]

'1位'

In [40]:
[(d[t[0]],t[1])for t in sorted(d.dfs.items(),key=lambda x:x[1],reverse=True)]

[('知る', 1461),
 ('人気', 1422),
 ('必要', 1395),
 ('関連リンク', 1371),
 ('考える', 1364),
 ('チェック', 1354),
 ('対応', 1338),
 ('現在', 1303),
 ('機能', 1278),
 ('感じる', 1273),
 ('行く', 1258),
 ('利用', 1257),
 ('声', 1255),
 ('出る', 1234),
 ('やる', 1213),
 ('聞く', 1206),
 ('しれる', 1204),
 ('映画', 1196),
 ('公開', 1171),
 ('世界', 1160),
 ('注目', 1132),
 ('語る', 1121),
 ('可能', 1105),
 ('仕事', 1098),
 ('良い', 1095),
 ('執筆', 1087),
 ('高い', 1078),
 ('わかる', 1068),
 ('smartphone', 1057),
 ('入る', 1029),
 ('くださる', 1010),
 ('開始', 1000),
 ('やすい', 990),
 ('受ける', 985),
 ('写真', 963),
 ('サービス', 962),
 ('強い', 959),
 ('ON', 954),
 ('予定', 933),
 ('続ける', 928),
 ('新しい', 927),
 ('作る', 920),
 ('搭載', 919),
 ('好き', 918),
 ('おく', 916),
 ('アプリ', 913),
 ('男性', 913),
 ('コメント', 908),
 ('内容', 907),
 ('選ぶ', 906),
 ('簡単', 901),
 ('使用', 892),
 ('もらう', 890),
 ('楽しむ', 888),
 ('開催', 887),
 ('max', 884),
 ('ユーザー', 876),
 ('違う', 867),
 ('エスマックス', 866),
 ('smaxjp', 863),
 ('すぎる', 861),
 ('入れる', 856),
 ('変わる', 853),
 ('女子', 848),
 ('最近', 847),
 ('問題', 843),
 

In [41]:
corpus = [d.doc2bow(w) for w in docs]

In [42]:
num_topics = 50

In [43]:
logging.basicConfig(format='%(message)s', level=logging.INFO)

In [44]:
test_size = int(len(corpus) * 0.1)
test_corpus = corpus[:test_size]
train_corpus= corpus[test_size:]
print("text size: ", len(test_corpus))
print("train size: ", len(train_corpus))

text size:  737
train size:  6639


In [45]:
lda = models.ldamodel.LdaModel(corpus=train_corpus, id2word=d, num_topics=num_topics,passes=10)
#lda = models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, passes=10)

using symmetric alpha at 0.02
using symmetric eta at 0.02
using serial LDA version on this node
running online (multi-pass) LDA training, 50 topics, 10 passes over the supplied corpus of 6639 documents, updating model once every 2000 documents, evaluating perplexity every 6639 documents, iterating 50x with a convergence threshold of 0.001000
PROGRESS: pass 0, at document #2000/6639
merging changes from 2000 documents into a model of 6639 documents
topic #8 (0.020): 0.003*"アプリ" + 0.003*"映画" + 0.003*"韓国" + 0.002*"考える" + 0.002*"肌" + 0.002*"行く" + 0.002*"人気" + 0.002*"シリーズ" + 0.002*"声" + 0.002*"対応"
topic #31 (0.020): 0.004*"smartphone" + 0.004*"max" + 0.004*"対応" + 0.004*"更新" + 0.004*"ソフトウェア" + 0.004*"アプリ" + 0.003*"表示" + 0.003*"画面" + 0.003*"搭載" + 0.003*"機能"
topic #13 (0.020): 0.004*"利用" + 0.003*"smartphone" + 0.003*"アプリ" + 0.003*"au" + 0.003*"対応" + 0.003*"ユーザー" + 0.003*"arrows" + 0.002*"監督" + 0.002*"画面" + 0.002*"仕事"
topic #41 (0.020): 0.007*"搭載" + 0.007*"対応" + 0.004*"映画" + 0.004*"公開" + 0.004*

topic #8 (0.020): 0.019*"声" + 0.019*"韓国" + 0.019*"ネット掲示板" + 0.014*"批判" + 0.011*"発言" + 0.009*"報じる" + 0.009*"ファン" + 0.009*"意見" + 0.009*"報道" + 0.008*"AKB48"
topic #5 (0.020): 0.008*"放送" + 0.008*"やる" + 0.006*"語る" + 0.006*"番組" + 0.006*"選手" + 0.006*"声" + 0.006*"コメント" + 0.005*"アナ" + 0.005*"中国" + 0.004*"blog"
topic #49 (0.020): 0.028*"ビデオSALON" + 0.013*"売れ筋" + 0.013*"チェック" + 0.011*"iPhone" + 0.009*"セキュリティ" + 0.009*"ソニー" + 0.007*"ユーザー" + 0.007*"スマホ" + 0.006*"ビデオ" + 0.005*"家電"
topic #6 (0.020): 0.016*"搭載" + 0.014*"対応" + 0.012*"カメラ" + 0.011*"NTTドコモ" + 0.011*"smartphone" + 0.009*"連続" + 0.009*"max" + 0.009*"cpu" + 0.008*"ディスプレイ" + 0.008*"Android Icecream Sandwich"
topic diff=1.287560, rho=0.397794
PROGRESS: pass 2, at document #4000/6639
merging changes from 2000 documents into a model of 6639 documents
topic #38 (0.020): 0.021*"選手" + 0.016*"語る" + 0.015*"sports" + 0.014*"試合" + 0.014*"監督" + 0.014*"watch" + 0.012*"チーム" + 0.011*"やる" + 0.008*"日本代表" + 0.008*"サッカー"
topic #30 (0.020): 0.017*"イベント" + 0.016

topic #45 (0.020): 0.017*"父" + 0.013*"親子" + 0.013*"息子" + 0.012*"感動" + 0.010*"絆" + 0.010*"家族" + 0.010*"リアル・スティール" + 0.010*"ロボット" + 0.010*"お父さん" + 0.009*"映画"
topic diff=0.307789, rho=0.346698
PROGRESS: pass 4, at document #4000/6639
merging changes from 2000 documents into a model of 6639 documents
topic #20 (0.020): 0.015*"ブランド" + 0.012*"デザイン" + 0.011*"アイテム" + 0.010*"モデル" + 0.010*"人気" + 0.008*"限定" + 0.008*"商品" + 0.006*"今年" + 0.006*"カラー" + 0.005*"税込"
topic #25 (0.020): 0.013*"ランキング" + 0.013*"書く" + 0.010*"文字" + 0.010*"年賀状" + 0.009*"捜査" + 0.009*"図" + 0.005*"上原" + 0.005*"名言" + 0.005*"石田" + 0.005*"履歴書"
topic #13 (0.020): 0.164*"ソフトバンク" + 0.043*"容量" + 0.041*"バッテリー" + 0.027*"ドコモ" + 0.018*"スマホ" + 0.013*"強い" + 0.012*"味方" + 0.011*"iPhone" + 0.010*"ケース" + 0.010*"大型"
topic #23 (0.020): 0.040*"結婚" + 0.015*"相手" + 0.012*"恋愛" + 0.011*"メール" + 0.011*"婚活" + 0.011*"男性" + 0.008*"占い" + 0.008*"出会い" + 0.007*"考える" + 0.007*"先生"
topic #11 (0.020): 0.017*"食べる" + 0.009*"作る" + 0.008*"野菜" + 0.008*"味" + 0.007*"料理" + 0

topic #0 (0.020): 0.021*"結婚" + 0.013*"子供" + 0.010*"夫" + 0.009*"子ども" + 0.008*"母親" + 0.008*"親" + 0.008*"独女" + 0.007*"生活" + 0.007*"母" + 0.007*"妻"
topic #29 (0.020): 0.017*"ゴルフ" + 0.010*"ゴルファー" + 0.009*"妄想" + 0.009*"クラブ" + 0.009*"ヘッド" + 0.008*"スピード" + 0.008*"スイング" + 0.007*"ドライバー" + 0.006*"ボール" + 0.006*"飛距離"
topic #25 (0.020): 0.014*"書く" + 0.012*"ランキング" + 0.011*"文字" + 0.010*"年賀状" + 0.009*"図" + 0.009*"捜査" + 0.005*"クラス" + 0.005*"上原" + 0.005*"名言" + 0.005*"履歴書"
topic diff=0.113955, rho=0.311294
PROGRESS: pass 6, at document #6000/6639
merging changes from 2000 documents into a model of 6639 documents
topic #30 (0.020): 0.024*"イベント" + 0.018*"クリスマス" + 0.015*"開催" + 0.012*"プレゼント" + 0.011*"参加" + 0.008*"チョコレート" + 0.008*"ホテル" + 0.008*"場所" + 0.007*"東京" + 0.007*"楽しむ"
topic #37 (0.020): 0.012*"line" + 0.010*"視聴率" + 0.010*"テレビ" + 0.010*"声" + 0.010*"人気" + 0.008*"カット" + 0.008*"チェック" + 0.007*"TV" + 0.006*"番組" + 0.006*"ネット"
topic #34 (0.020): 0.024*"動画" + 0.021*"視聴" + 0.021*"番組" + 0.019*"録画" + 0.019*"配信" + 0.

merging changes from 2000 documents into a model of 6639 documents
topic #49 (0.020): 0.028*"ビデオSALON" + 0.024*"チェック" + 0.022*"売れ筋" + 0.017*"iPhone" + 0.013*"家電" + 0.011*"ソニー" + 0.009*"アップル" + 0.008*"スマホ" + 0.008*"パナソニック" + 0.007*"ニュース"
topic #30 (0.020): 0.024*"イベント" + 0.018*"クリスマス" + 0.015*"開催" + 0.012*"プレゼント" + 0.011*"参加" + 0.008*"ホテル" + 0.008*"チョコレート" + 0.008*"場所" + 0.008*"東京" + 0.008*"体験"
topic #27 (0.020): 0.031*"ファッション" + 0.023*"着る" + 0.020*"スタイル" + 0.020*"シャツ" + 0.017*"色" + 0.016*"スーツ" + 0.013*"服" + 0.011*"Paul Smith" + 0.011*"新生活" + 0.009*"コレクション"
topic #35 (0.020): 0.026*"PC" + 0.019*"快適" + 0.015*"システム" + 0.014*"セキュリティ" + 0.014*"windows" + 0.013*"ファイル" + 0.011*"メモリー" + 0.009*"会議" + 0.008*"ツール" + 0.007*"解消"
topic #31 (0.020): 0.018*"機能" + 0.017*"smartphone" + 0.017*"画面" + 0.015*"max" + 0.011*"操作" + 0.010*"livedoor Blog" + 0.010*"表示" + 0.008*"やすい" + 0.008*"設定" + 0.008*"キー"
topic diff=0.063227, rho=0.284907
-8.206 per-word bound, 295.2 perplexity estimate based on a held-out cor

In [46]:
def get_topic_words(topic_id):
    for t in lda.get_topic_terms(topic_id):
        print("{}: {}".format(d[t[0]], t[1]))

In [47]:
def get_topic_words(topic_id):
    for t in lda.get_topic_terms(topic_id):
        print("{}: {}".format(d[t[0]], t[1]))
for t in range(10):
    print("Topic # ",t)
    get_topic_words(t)
    print("\n")

Topic #  0
結婚: 0.020758062601089478
子供: 0.013362914323806763
子ども: 0.010901467874646187
夫: 0.009536555968225002
親: 0.008387825451791286
生活: 0.00789244007319212
独女: 0.007785802707076073
母親: 0.007341646123677492
母: 0.006530567072331905
妻: 0.0062169902957975864


Topic #  1
バイト: 0.015547038987278938
アベンジャーズ: 0.014125838875770569
萌える: 0.011982954107224941
クーラー: 0.010940805077552795
キャプテン・アメリカ: 0.009378809481859207
ビクター: 0.00896498467773199
漫画家: 0.00863814726471901
大島優子: 0.008028542622923851
闇金ウシジマくん: 0.00720768328756094
グラフィック: 0.006690305657684803


Topic #  2
SSD: 0.013922677375376225
エリア: 0.009531128220260143
パソコン: 0.00935115572065115
PC: 0.008194982074201107
生産: 0.008189031854271889
節電: 0.007790529169142246
接続: 0.007780350279062986
電力: 0.007540238089859486
サービス: 0.007476441562175751
インテル: 0.007244651671499014


Topic #  3
心: 0.00867356825619936
人生: 0.006939167622476816
続ける: 0.006298606283962727
強い: 0.005809150170534849
感じる: 0.005743558518588543
相手: 0.005382594186812639
気持ち: 0.0052753365

In [48]:
topics = sorted(lda.get_document_topics(corpus[0]), key=lambda t:t[1], reverse=True)
for t in topics[:10]:
    print("{}: {}".format(t[0], t[1]))

26: 0.5254321098327637
10: 0.1630166918039322
3: 0.11774574965238571
23: 0.08206278830766678
18: 0.04296118766069412
44: 0.03669465333223343
29: 0.016388626769185066


In [49]:
for t in topics[:10]:
    print("Topic # ",t[0])
    get_topic_words(t[0])
    print("\n")

Topic #  26
恋愛: 0.014350672252476215
男性: 0.011099313385784626
女子: 0.010217814706265926
読む: 0.009644685313105583
好き: 0.007067806087434292
恋: 0.007065121084451675
食べる: 0.0059796408750116825
Peachy: 0.005232261493802071
セックス: 0.00497244531288743
トイレ: 0.004518234636634588


Topic #  10
男性: 0.01951467990875244
聞く: 0.014128956012427807
好き: 0.009679840877652168
仕事: 0.009660676121711731
話す: 0.009185049682855606
感じる: 0.008436602540314198
行く: 0.008031627163290977
独女: 0.006659291218966246
しれる: 0.006586574018001556
友達: 0.006513142958283424


Topic #  3
心: 0.00867356825619936
人生: 0.006939167622476816
続ける: 0.006298606283962727
強い: 0.005809150170534849
感じる: 0.005743558518588543
相手: 0.005382594186812639
気持ち: 0.005275336559861898
知る: 0.005037281662225723
生きる: 0.005006181541830301
夢: 0.004784357734024525


Topic #  23
結婚: 0.034647297114133835
相手: 0.019656432792544365
恋愛: 0.01482250727713108
男性: 0.0145696010440588
メール: 0.013400928117334843
婚活: 0.009968243539333344
考える: 0.008262764662504196
出会い: 0.0080631

In [50]:
dm.read_doc(0)

http://news.livedoor.com/article/detail/6755396/
2012-07-15T18:00:00+0900
男が好む清純派の演じ方／「結婚はムリ」と男に見切りをつけた瞬間など−【恋愛】週間ランキング 
Peachyでも大人気の「恋愛」をテーマにした記事の週間ランキングです！2012年7月5日〜7月11日の間にPeachyのアプリでみんなが読んだ恋愛記事TOP5をお届けします。

第1位：男はビッチよりもピュアが好き！清純派っぽく演じる方法8選

男性は、基本的に自分より恋愛経験値の低い女性を好む。著者も最近出会った男性から、これまで付き合った人数を訊かれ「うーん、2人かな」と答えてみると「26歳で2人！　意外と少ないんだね」とニンマリされ、かなり喜ばれてしまった。やはりピュアなほうがモテる。
そこで、男性への会話のなかで“ピュアっぽく見せる”テクニックをご紹介しよう。

第2位：「この男とは絶対結婚できない！」彼氏と別れを決意した瞬間5選

女性が彼氏に見切りをつけるときとは、どんなときなのでしょうか？　筆者の周りにいる多くの女性は、結婚を考え出したときという意見でした。
そこで、「この男とは、絶対に結婚できない！」と思う条件を聞いてみました。

第3位：彼氏とのマンネリを引き起こす女性7の行動

最近彼からの愛情を昔より感じなくなった…と嘆き節な女性、その原因考えたことありますか？
今回は、そんなマンネリ状態を引き起こす女性の行動をまとめてみました。

第4位：許されるのは綾瀬はるかだけ！男が幻滅する「干物女」の特徴6つ

ドラマや映画がヒットした『ホタルノヒカリ』。おかげで干物女という言葉が一般化しましたが、そんな今でも、男性は女性に対してきちんとしていて欲しいと思っているようです。
オーディション等で女性を見ぬかなければいけない機会の多い、放送作家の白武ときおさんは以下のように語ります。「男性は意外と細かいところも見ています。だらしない女の人が好きな男性はまずいないので気を抜くと大変なことになりますよ」

第5位：男性に聞く！　「絶対裏切れない」と感じた彼女の行動

付き合っている人に、浮気をされたら……。余計な心配とはわかっていても、つい疑心暗鬼になって「浮気しないでね」なんて言っては彼を困らせていませんか？　彼らが「

## perplexity

In [51]:
N = sum(count for doc in test_corpus for id, count in doc)
print("N: ",N)

N:  137442


In [52]:
lda.log_perplexity(test_corpus)

-10.778 per-word bound, 1755.3 perplexity estimate based on a held-out corpus of 737 documents with 137442 words


-10.777508369973425

Estimate the variational bound of documents from corpus: E_q[log p(corpus)] - E_q[log q(corpus)]

In [53]:
bound = lda.bound(test_corpus)/N
bound 

-10.777452194049392

In [54]:
perplexity = np.exp2(-bound ) # 2^(-bound per words)

In [55]:
perplexity = np.exp2(-lda.log_perplexity(test_corpus))

-10.777 per-word bound, 1755.1 perplexity estimate based on a held-out corpus of 737 documents with 137442 words


In [56]:
print("perplexity:", perplexity)

perplexity: 1755.0953243233762


## see on tensorboard

http://projector.tensorflow.org/  
ref: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Tensorboard_visualizations.ipynb


In [57]:
all_topics = lda.get_document_topics(corpus, minimum_probability=0)

In [58]:
all_topics[0]

[(0, 0.00013698632),
 (1, 0.00013698632),
 (2, 0.00013698632),
 (3, 0.11780274),
 (4, 0.00013698632),
 (5, 0.00013698632),
 (6, 0.00013698632),
 (7, 0.00013698632),
 (8, 0.00013698632),
 (9, 0.00013698632),
 (10, 0.1629944),
 (11, 0.00013698632),
 (12, 0.00013698632),
 (13, 0.00013698632),
 (14, 0.00013698632),
 (15, 0.00013698632),
 (16, 0.00013698632),
 (17, 0.00013698632),
 (18, 0.043011263),
 (19, 0.00013698632),
 (20, 0.00013698632),
 (21, 0.00013698632),
 (22, 0.00013698632),
 (23, 0.0820642),
 (24, 0.00013698632),
 (25, 0.00013698632),
 (26, 0.52544063),
 (27, 0.00013698632),
 (28, 0.00013698632),
 (29, 0.016388437),
 (30, 0.00013698632),
 (31, 0.00013698632),
 (32, 0.00013698632),
 (33, 0.00013698632),
 (34, 0.00013698632),
 (35, 0.00013698632),
 (36, 0.00013698632),
 (37, 0.00013698632),
 (38, 0.00013698632),
 (39, 0.00013698632),
 (40, 0.00013698632),
 (41, 0.00013698632),
 (42, 0.00013698632),
 (43, 0.009850103),
 (44, 0.036694784),
 (45, 0.00013698632),
 (46, 0.00013698632)

In [59]:
with open('doc_lda_tensor.tsv','w') as w:
    for doc_topics in all_topics:
        for topics in doc_topics:
            w.write(str(topics[1])+ "\t")
        w.write("\n")    

In [60]:
meta = [str(a).split("/") for a in articles]

In [61]:
meta[2]

['text', 'dokujo-tsushin', 'dokujo-tsushin-6313075.txt']

In [62]:
with open('doc_lda_metadata.tsv','w') as w:
    w.write('Titles\tGenres\n')
    for m in meta:
        w.write("%s\t%s\n" % (m[1].split("-")[0], m[1]))

## 類似度

In [63]:
doc_topics = np.array([lda[c] for c in corpus])

In [64]:
doc_topics.shape

(7376, 50, 2)

In [65]:
features = np.zeros( (len(doc_topics), num_topics), float)

for doc_id, t in enumerate(doc_topics):
    for topic_id, weight in t:
        features[doc_id, int(topic_id)] = weight

例えば
$$
\text{doc_id(7376)}
\underbrace{
\begin{pmatrix}
0 & 0 & weight & \cdots & 0\\
0 & 0 & 0 & \cdots & 0\\
0 & weight & 0 & \cdots & 0\\
\vdots & \vdots & \vdots & \vdots & \vdots\\
0 & 0 & 0 & \cdots & 0
\end{pmatrix}
}_{\text{topics(50)}}
$$

In [66]:
from scipy.spatial import distance
pairwise = distance.squareform(distance.pdist(features, metric="cosine"))

$$
1 - \frac{u \cdot v}
         {{||u||}_2 {||v||}_2}
$$

In [67]:
def similar_to(doc_id, num=5, distance=False):
    doc = pairwise[doc_id]
    ids = np.argsort(doc)[1: num+1] # the most similar is self.
    if distance:
        dist = np.sort(doc)[1: num+1]
        return [(x,y) for x, y in zip(ids, dist)]
    else:
        return ids

In [68]:
similar_to(0, num=10, distance=True)

[(7295, 0.012341012535360463),
 (6102, 0.02562400191854275),
 (38, 0.030022621721828457),
 (2453, 0.03457848070898806),
 (225, 0.04141893209719927),
 (6403, 0.04293648170997866),
 (4701, 0.045845680150193635),
 (3590, 0.05427043041976365),
 (4943, 0.059301586726990485),
 (3232, 0.06045657039498742)]

In [69]:
from gensim import similarities

In [70]:
doc_index = similarities.docsim.MatrixSimilarity(lda[corpus])

scanning corpus to determine the number of features (consider setting `num_features` explicitly)
creating matrix with 7376 documents and 50 features


In [71]:
len(doc_index)

7376

In [72]:
c = corpus[0]
vec_lda = lda[c]
vec_lda

[(0, 0.00013698632),
 (1, 0.00013698632),
 (2, 0.00013698632),
 (3, 0.117837355),
 (4, 0.00013698632),
 (5, 0.00013698632),
 (6, 0.00013698632),
 (7, 0.00013698632),
 (8, 0.00013698632),
 (9, 0.00013698632),
 (10, 0.16303116),
 (11, 0.00013698632),
 (12, 0.00013698632),
 (13, 0.00013698632),
 (14, 0.00013698632),
 (15, 0.00013698632),
 (16, 0.00013698632),
 (17, 0.00013698632),
 (18, 0.043045387),
 (19, 0.00013698632),
 (20, 0.00013698632),
 (21, 0.00013698632),
 (22, 0.00013698632),
 (23, 0.082052104),
 (24, 0.00013698632),
 (25, 0.00013698632),
 (26, 0.52541196),
 (27, 0.00013698632),
 (28, 0.00013698632),
 (29, 0.016388318),
 (30, 0.00013698632),
 (31, 0.00013698632),
 (32, 0.00013698632),
 (33, 0.00013698632),
 (34, 0.00013698632),
 (35, 0.00013698632),
 (36, 0.00013698632),
 (37, 0.00013698632),
 (38, 0.00013698632),
 (39, 0.00013698632),
 (40, 0.00013698632),
 (41, 0.00013698632),
 (42, 0.00013698632),
 (43, 0.009784975),
 (44, 0.036695372),
 (45, 0.00013698632),
 (46, 0.00013698

In [73]:
s = doc_index.__getitem__(vec_lda) # There is same method "get_similarities" but it is said no use in the code.

In [74]:
s = sorted(enumerate(s), key=lambda t: t[1], reverse=True)

In [75]:
s[:10]

[(0, 1.0),
 (7295, 0.987645),
 (6102, 0.9743256),
 (38, 0.9699433),
 (2453, 0.9653756),
 (225, 0.95853686),
 (6403, 0.95696986),
 (4701, 0.95548356),
 (3590, 0.9456991),
 (4943, 0.9406523)]

In [76]:
for doc_id, sim in s[1:3]:
    with articles[doc_id].open() as f:
        print("#", doc_id)
        print(f.read())
        print("===========")

# 7295
http://news.livedoor.com/article/detail/6732527/
2012-07-08T17:00:00+0900
美人は年をとるとモテなくなる／男の本心を見抜くテクなど−【恋愛】週間ランキング
Peachyでも大人気の「恋愛」をテーマにした記事の週間ランキングです！2012年6月28日〜7月4日の間にPeachyのアプリでみんなが読んだ恋愛記事TOP5をお届けします。

第1位：彼の本心を一瞬で見抜けるテク

「彼って無口で、何を考えているのかわかんない…。でも、あのアンニュイな感じが魅力的〜」なんて思って付き合ってみたはいいものの、彼から発される負のオーラになんだかこっちもどんより…。そんなネガティブ男を捕まえちゃう前に、「風水芸人◆出雲阿国」サイトでご紹介している風水で、彼がネガティブかどうか、探りを入れちゃいましょう！

第2位：男が本気でドン引きしている「痛い女のメイク」まとめ

「日本女性のメイクは整形並み」なんて言われるほど、化粧は女性にとってキレイになるために欠かせないもののひとつです。
しかし中には男性から見ると「ちょっとやり過ぎでは？」と思われてしまうメイクがあるようです。

第3位：彼氏とはあなたを映す○○○

「彼氏と穏やかな関係でいたいのに、いつもケンカばっかり…」と悩むことってありませんか？　そんな女子に向けて、「大人の恋愛偏差値」サイトの監修者・池内ひろ美さんは次のようにアドバイスしています。

第4位：美人は歳をとるとモテなくなる!?男が興味を示さない美人の特徴

先日、居酒屋で男女7人の友人で食事をしていると、隣から女子会をしている声が聞こえてきました。5人組みの女子会グループは、どのひともオシャレな美人系。なのに友人の男性陣はチラッと見ただけで興味がない様子。しかし、次にお店に入ってきた、どこにでもいそうな可もなく不可もない中レベルの女子会グループには興味津々。なぜ、美人を差し置いて、中レベル女子がモテるという状況になるのでしょうか。

第5位：「セカンドバージン」から抜け出せない女性に捧ぐ、前向きな言葉

鈴木京香が主演したドラマ『セカンドバージン』（NHK）がきっかけで、タイトルのこの言葉は一躍話題となりました。これは、処女ではないけれど、セックスがごぶさたになっている女性のことを

In [77]:
d.save_as_text("dict.txt")
corpora.MmCorpus.serialize("cop.mm", corpus)
lda.save("lda.model")
doc_index.save("sim")

saving dictionary mapping to dict.txt
storing corpus in Matrix Market format to cop.mm
saving sparse matrix to cop.mm
PROGRESS: saving document #0
PROGRESS: saving document #1000
PROGRESS: saving document #2000
PROGRESS: saving document #3000
PROGRESS: saving document #4000
PROGRESS: saving document #5000
PROGRESS: saving document #6000
PROGRESS: saving document #7000
saved 7376x23513 matrix, density=0.522% (904999/173431888)
saving MmCorpus index to cop.mm.index
saving LdaState object under lda.model.state, separately None
saved lda.model.state
saving LdaModel object under lda.model, separately ['expElogbeta', 'sstats']
storing np array 'expElogbeta' to lda.model.expElogbeta.npy
not storing attribute id2word
not storing attribute dispatcher
not storing attribute state
saved lda.model
saving MatrixSimilarity object under sim, separately None
saved sim


In [78]:
from gensim import models, corpora, similarities

corpus = corpora.MmCorpus("cop.mm")
lda = models.ldamodel.LdaModel.load("lda.model")
d = corpora.Dictionary.load_from_text("dict.txt")
doc_index = similarities.docsim.MatrixSimilarity.load("sim")

loaded corpus index from cop.mm.index
initializing cython corpus reader from cop.mm
accepted corpus with 7376 documents, 23513 features, 904999 non-zero entries
loading LdaModel object from lda.model
loading expElogbeta from lda.model.expElogbeta.npy with mmap=None
setting ignored attribute id2word to None
setting ignored attribute dispatcher to None
setting ignored attribute state to None
loaded lda.model
loading LdaState object from lda.model.state
loaded lda.model.state
loading MatrixSimilarity object from sim
loaded sim


In [79]:
doc_index.__getitem__(lda[corpus[0]])

array([1.        , 0.584555  , 0.34240198, ..., 0.12095422, 0.38471815,
       0.01057947], dtype=float32)